In [1]:
#Imports 

import re
import pandas as pd
import os
import gensim
from gensim.utils import simple_preprocess
from gensim.corpora.dictionary import Dictionary
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /Users/nata/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Loading raw data

In [4]:
all_articles = pd.read_csv('french_data.csv')

In [8]:
all_articles.head()

,_id,url,date,text,title
0,5ec760320e5c92500d004aee,https://plus.lesoir.be/302282/article/2020-05-...,2020-05-22T04:22:52.000Z,Les concentrations de différents polluants rel...,La qualité de l’air en nette amélioration à Br...
1,5ec7603216193c3860cf1616,https://www.rtbf.be/info/regions/detail_corona...,2020-05-22T04:37:03.000Z,L’amélioration de la qualité de l’air bruxello...,Coronavirus à Bruxelles : on respire mieux grâ...
2,5ec76032039b233f9642aeb6,https://plus.lesoir.be/302121/article/2020-05-...,2020-05-20T19:36:26.000Z,La résolution pose plusieurs principes tel que...,La Chambre vote une résolution encadrant l’usa...
3,5ec760320e5c92500d004af3,https://plus.lesoir.be/302273/article/2020-05-...,2020-05-21T21:01:32.000Z,Le nouveau coronavirus a officiellement touché...,Plus de cinq millions de cas de Covid-19 dans ...
4,5ec7603216193c3860cf161f,https://www.rtbf.be/info/economie/detail_coron...,2020-05-22T04:00:00.000Z,Les résultats d’une enquête menée par le prest...,Coronavirus en Belgique : les ouvriers plus to...


In [6]:
all_articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58614 entries, 0 to 58613
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   _id     58614 non-null  object
 1   url     58614 non-null  object
 2   date    58614 non-null  object
 3   text    58614 non-null  object
 4   title   58614 non-null  object
dtypes: object(5)
memory usage: 2.2+ MB


Cleaning duplicates

In [12]:
len(all_articles['text'].unique())

45159

In [14]:
unique_articles = all_articles.drop_duplicates(subset=['text'])


In [15]:
len(unique_articles)

45159

Checking languages

In [16]:
from langdetect import detect_langs

In [33]:
languages = []
for row in range(len(unique_articles)):
    languages.append(detect_langs(unique_articles.iloc[row, 3]))

In [34]:
str(languages[1]).split(':')[0][1:]

'fr'

In [35]:
langs = [str(lang).split(':')[0][1:] for lang in languages]

In [36]:
langs_set = set(langs)

In [38]:
unique_articles['langs'] = langs

/var/folders/zz/5wlqs6xs1356t872c7mj3b_40000gn/T/ipykernel_3946/3910387253.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_articles['langs'] = langs


In [51]:
articles = unique_articles[unique_articles['langs']== 'fr']

In [53]:
unique_articles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45159 entries, 0 to 58613
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   _id     45159 non-null  object
 1   url     45159 non-null  object
 2   date    45159 non-null  object
 3   text    45159 non-null  object
 4   title   45159 non-null  object
 5   langs   45159 non-null  object
dtypes: object(6)
memory usage: 2.4+ MB


In [52]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45158 entries, 0 to 58613
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   _id     45158 non-null  object
 1   url     45158 non-null  object
 2   date    45158 non-null  object
 3   text    45158 non-null  object
 4   title   45158 non-null  object
 5   langs   45158 non-null  object
dtypes: object(6)
memory usage: 2.4+ MB


In [55]:
articles.to_csv('clean_french_data.csv')

Preprocessing

In [2]:
french_stopwords = stopwords.words('french')

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc= True))

def remove_stopwords(articles):
    return[[word for word in simple_preprocess(str(doc)) if word not in french_stopwords] for doc in articles]